# 🔬 Калибровка генератора на реальных данных

Этот notebook анализирует реальные данные скважин и извлекает параметры для генератора синтетических скважин.

## 📋 Что делает этот notebook:
1. Загружает реальные данные скважин
2. Анализирует частоты встречаемости литологий
3. Вычисляет длины литологических серий
4. Извлекает физические свойства пород
5. Строит матрицу переходов между литологиями
6. Оценивает уровень шума и эффекты инверсии
7. Генерирует конфигурационный файл для синтетического генератора

## 📊 Требуемый формат данных:
CSV файл с колонками:
- `Depth` - глубина (м)
- `Lithology` - литология (текст)
- `Gamma` - гамма-каротаж (API) [опционально]
- `Rho` - плотность (г/см³) [опционально]
- `Porosity` - пористость [опционально]
- `Sw` - водонасыщенность [опционально]

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Библиотеки загружены!")

## 2. Загрузка данных

Укажите путь к вашему CSV файлу с данными реальной скважины:

In [ ]:
# ИЗМЕНИТЕ ПУТЬ К ВАШЕМУ ФАЙЛУ
input_file = 'real_well_data.csv'

# Загрузка данных
try:
    df = pd.read_csv(input_file)
    print(f"✅ Данные загружены: {df.shape[0]} строк × {df.shape[1]} столбцов")
    print(f"\nКолонки: {', '.join(df.columns)}")
    print(f"\nПервые 5 строк:")
    display(df.head())
except FileNotFoundError:
    print(f"❌ Файл {input_file} не найден!")
    print("\n💡 Создаем ДЕМО-данные для примера...")
    
    # Создание демо-данных
    np.random.seed(42)
    n = 500
    
    # Простая последовательность литологий
    liths = ['sand', 'shale', 'siltstone', 'coal']
    lithology = []
    for _ in range(50):
        lith = np.random.choice(liths, p=[0.2, 0.5, 0.2, 0.1])
        length = np.random.randint(5, 15)
        lithology.extend([lith] * length)
    lithology = lithology[:n]
    
    df = pd.DataFrame({
        'Depth': np.linspace(1000, 1500, n),
        'Lithology': lithology,
        'Gamma': [40 + np.random.normal(0, 5) if l == 'sand' else 
                  70 + np.random.normal(0, 5) if l == 'shale' else
                  60 + np.random.normal(0, 5) if l == 'siltstone' else
                  30 + np.random.normal(0, 5) for l in lithology],
        'Rho': [2.3 + np.random.normal(0, 0.05) if l == 'sand' else 
                2.4 + np.random.normal(0, 0.05) if l == 'shale' else
                2.5 + np.random.normal(0, 0.05) if l == 'siltstone' else
                1.5 + np.random.normal(0, 0.05) for l in lithology],
        'Porosity': [0.25 + np.random.normal(0, 0.03) if l == 'sand' else 
                     0.15 + np.random.normal(0, 0.02) if l == 'shale' else
                     0.18 + np.random.normal(0, 0.02) if l == 'siltstone' else
                     0.35 + np.random.normal(0, 0.04) for l in lithology],
        'Sw': [0.3 + np.random.normal(0, 0.05) if l == 'sand' else 
               0.8 + np.random.normal(0, 0.05) if l == 'shale' else
               0.6 + np.random.normal(0, 0.05) if l == 'siltstone' else
               0.2 + np.random.normal(0, 0.05) for l in lithology]
    })
    
    print("✅ ДЕМО-данные созданы")
    display(df.head())
    print("\n⚠️ Замените 'real_well_data.csv' на путь к вашим данным!")

## 3. Анализ частот встречаемости литологий

In [ ]:
print("="*70)
print("1️⃣  ЧАСТОТЫ ВСТРЕЧАЕМОСТИ ЛИТОЛОГИЙ")
print("="*70)

counts = df['Lithology'].value_counts()
frequencies = counts / len(df)

# Создание диапазонов ±20%
freq_range = {}
for lith in df['Lithology'].unique():
    freq = frequencies[lith]
    freq_range[lith] = (max(0.01, freq * 0.8), min(1.0, freq * 1.2))

# Вывод результатов
print("\nЛитология           Точек    Частота    Диапазон для генератора")
print("-" * 70)
for lith in df['Lithology'].unique():
    freq = frequencies[lith]
    print(f"{lith:20s} {counts[lith]:5d}    {freq*100:5.2f}%    ({freq_range[lith][0]:.3f}, {freq_range[lith][1]:.3f})")

# Визуализация
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(range(len(counts)), counts.values, color='steelblue', edgecolor='black')
ax1.set_xticks(range(len(counts)))
ax1.set_xticklabels(counts.index, rotation=45, ha='right')
ax1.set_ylabel('Количество точек')
ax1.set_title('Распределение литологий')
ax1.grid(alpha=0.3, axis='y')

# Добавление процентов
for i, (count, lith) in enumerate(zip(counts.values, counts.index)):
    percentage = (count / len(df)) * 100
    ax1.text(i, count + 5, f'{percentage:.1f}%', ha='center', va='bottom', fontweight='bold')

ax2.pie(counts.values, labels=counts.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Процентное соотношение')

plt.tight_layout()
plt.show()

# Сохранение в конфиг
config = {'lithology_freq_range': freq_range}

## 4. Анализ длин литологических серий

In [ ]:
print("="*70)
print("2️⃣  ДЛИНЫ ЛИТОЛОГИЧЕСКИХ СЕРИЙ")
print("="*70)

# Анализ серий
series_lengths = defaultdict(list)

current_lith = df['Lithology'].iloc[0]
current_length = 1

for lith in df['Lithology'].iloc[1:]:
    if lith == current_lith:
        current_length += 1
    else:
        series_lengths[current_lith].append(current_length)
        current_lith = lith
        current_length = 1

series_lengths[current_lith].append(current_length)

# Статистика
series_range = {}
print("\nЛитология            Min    Max    Mean   Median  Кол-во серий")
print("-" * 70)

for lith in df['Lithology'].unique():
    if lith in series_lengths and len(series_lengths[lith]) > 0:
        lengths = series_lengths[lith]
        min_len = int(min(lengths))
        max_len = int(max(lengths))
        mean_len = np.mean(lengths)
        median_len = np.median(lengths)
        
        series_range[lith] = (min_len, max_len)
        
        print(f"{lith:20s} {min_len:4d}   {max_len:4d}   {mean_len:6.1f}   {median_len:6.1f}   {len(lengths):4d}")

# Визуализация распределения длин
fig, axes = plt.subplots(1, len(series_lengths), figsize=(15, 4))
if len(series_lengths) == 1:
    axes = [axes]

for ax, (lith, lengths) in zip(axes, series_lengths.items()):
    ax.hist(lengths, bins=20, color='coral', edgecolor='black', alpha=0.7)
    ax.set_xlabel('Длина серии (точек)')
    ax.set_ylabel('Частота')
    ax.set_title(f'{lith}')
    ax.grid(alpha=0.3)
    ax.axvline(np.mean(lengths), color='red', linestyle='--', label=f'Mean: {np.mean(lengths):.1f}')
    ax.legend()

plt.suptitle('Распределение длин серий по литологиям', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Добавление в конфиг
config['series_length_range'] = series_range

## 5. Физические свойства пород

In [ ]:
print("="*70)
print("3️⃣  ФИЗИЧЕСКИЕ СВОЙСТВА ПОРОД")
print("="*70)

properties = {}

for lith in df['Lithology'].unique():
    lith_data = df[df['Lithology'] == lith]
    props = {}
    
    print(f"\n{lith}:")
    
    # Пористость
    if 'Porosity' in df.columns:
        phi_mean = lith_data['Porosity'].mean()
        phi_std = lith_data['Porosity'].std()
        props['porosity'] = (float(phi_mean), float(phi_std))
        print(f"  Пористость:        {phi_mean:.3f} ± {phi_std:.3f}")
    
    # Водонасыщенность
    if 'Sw' in df.columns:
        sw_mean = lith_data['Sw'].mean()
        sw_std = lith_data['Sw'].std()
        props['sw'] = (float(sw_mean), float(sw_std))
        print(f"  Sw:                {sw_mean:.3f} ± {sw_std:.3f}")
    
    # ГК
    if 'Gamma' in df.columns:
        gamma_mean = lith_data['Gamma'].mean()
        gamma_std = lith_data['Gamma'].std()
        props['gamma_base'] = float(gamma_mean)
        props['gamma_std'] = float(gamma_std)
        print(f"  ГК:                {gamma_mean:.1f} ± {gamma_std:.1f} API")
    
    # Плотность
    if 'Rho' in df.columns:
        rho_mean = lith_data['Rho'].mean()
        rho_std = lith_data['Rho'].std()
        props['observed_density'] = (float(rho_mean), float(rho_std))
        print(f"  Плотность:         {rho_mean:.2f} ± {rho_std:.2f} г/см³")
        
        # Матричная плотность
        if 'Porosity' in df.columns:
            phi_avg = lith_data['Porosity'].mean()
            rho_fluid = 1.0
            if phi_avg < 0.99:
                rho_matrix = (rho_mean - phi_avg * rho_fluid) / (1 - phi_avg)
                props['matrix_density'] = float(rho_matrix)
                print(f"  Матр. плотность:   {rho_matrix:.2f} г/см³ (расчет)")
    
    # Цвет для визуализации
    color_map = {
        'sand': 'yellow', 'shale': 'brown', 'carbonate': 'orange',
        'carbonate_sand': 'orange', 'coal': 'black', 'siltstone': 'gray',
        'limestone': 'lightblue', 'sandstone': 'gold'
    }
    props['color'] = color_map.get(lith.lower(), 'gray')
    
    properties[lith] = props

config['lithology_properties'] = properties

## 6. Box plots по литологиям

In [ ]:
# Определение доступных параметров
params = []
if 'Porosity' in df.columns:
    params.append('Porosity')
if 'Sw' in df.columns:
    params.append('Sw')
if 'Gamma' in df.columns:
    params.append('Gamma')
if 'Rho' in df.columns:
    params.append('Rho')

if len(params) > 0:
    n_params = len(params)
    fig, axes = plt.subplots(1, n_params, figsize=(5*n_params, 5))
    if n_params == 1:
        axes = [axes]
    
    for ax, param in zip(axes, params):
        df.boxplot(column=param, by='Lithology', ax=ax)
        ax.set_xlabel('Литология')
        ax.set_ylabel(param)
        ax.set_title(f'{param} по литологиям')
        ax.get_figure().suptitle('')
    
    plt.tight_layout()
    plt.show()

## 7. Матрица переходов между литологиями

In [ ]:
print("="*70)
print("4️⃣  МАТРИЦА ПЕРЕХОДОВ МЕЖДУ ЛИТОЛОГИЯМИ")
print("="*70)

# Расчет матрицы переходов
lithology_sequence = df['Lithology'].tolist()
unique_liths = sorted(set(lithology_sequence))
n = len(unique_liths)
lith_to_idx = {lith: i for i, lith in enumerate(unique_liths)}

transition_counts = np.zeros((n, n))
for i in range(len(lithology_sequence) - 1):
    from_idx = lith_to_idx[lithology_sequence[i]]
    to_idx = lith_to_idx[lithology_sequence[i + 1]]
    transition_counts[from_idx, to_idx] += 1

# Нормализация
row_sums = transition_counts.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
transition_matrix = transition_counts / row_sums

# Вывод
print("\nВероятности переходов:")
df_trans = pd.DataFrame(transition_matrix, index=unique_liths, columns=unique_liths)
print(df_trans.round(3))

# Визуализация тепловой карты
plt.figure(figsize=(10, 8))
sns.heatmap(transition_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=unique_liths, yticklabels=unique_liths,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Матрица переходов между литологиями', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('К литологии')
plt.ylabel('От литологии')
plt.tight_layout()
plt.show()

# Добавление в конфиг
config['transition_matrix'] = transition_matrix.tolist()
config['lithology_states'] = unique_liths

## 8. Каротажные кривые

In [ ]:
if 'Depth' in df.columns:
    n_curves = sum([1 for col in ['Gamma', 'Rho', 'Porosity', 'Sw'] if col in df.columns])
    
    if n_curves > 0:
        fig, axes = plt.subplots(1, n_curves, figsize=(4*n_curves, 10), sharey=True)
        if n_curves == 1:
            axes = [axes]
        
        idx = 0
        
        if 'Gamma' in df.columns:
            axes[idx].plot(df['Gamma'], df['Depth'], 'g-', linewidth=0.5)
            axes[idx].set_xlabel('ГК (API)')
            axes[idx].set_ylabel('Глубина (м)')
            axes[idx].set_title('Гамма-каротаж')
            axes[idx].invert_yaxis()
            axes[idx].grid(alpha=0.3)
            idx += 1
        
        if 'Rho' in df.columns:
            axes[idx].plot(df['Rho'], df['Depth'], 'b-', linewidth=0.5)
            axes[idx].set_xlabel('Плотность (г/см³)')
            axes[idx].set_title('Плотность')
            axes[idx].grid(alpha=0.3)
            idx += 1
        
        if 'Porosity' in df.columns:
            axes[idx].plot(df['Porosity'], df['Depth'], 'orange', linewidth=0.5)
            axes[idx].set_xlabel('Пористость')
            axes[idx].set_title('Пористость')
            axes[idx].grid(alpha=0.3)
            idx += 1
        
        if 'Sw' in df.columns:
            axes[idx].plot(df['Sw'], df['Depth'], 'cyan', linewidth=0.5)
            axes[idx].set_xlabel('Sw')
            axes[idx].set_title('Водонасыщенность')
            axes[idx].grid(alpha=0.3)
        
        plt.suptitle('Каротажные кривые реальной скважины', fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()

## 9. Кроссплоты

In [ ]:
if 'Gamma' in df.columns and 'Rho' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Gamma vs Rho по литологии
    for lith in df['Lithology'].unique():
        lith_data = df[df['Lithology'] == lith]
        axes[0].scatter(lith_data['Gamma'], lith_data['Rho'], 
                       label=lith, alpha=0.6, s=30)
    axes[0].set_xlabel('ГК (API)')
    axes[0].set_ylabel('Плотность (г/см³)')
    axes[0].set_title('Gamma vs Density (по литологии)')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Gamma vs Rho по пористости (если есть)
    if 'Porosity' in df.columns:
        sc = axes[1].scatter(df['Gamma'], df['Rho'], c=df['Porosity'], 
                            cmap='viridis', alpha=0.7, s=30)
        axes[1].set_xlabel('ГК (API)')
        axes[1].set_ylabel('Плотность (г/см³)')
        axes[1].set_title('Gamma vs Density (по пористости)')
        axes[1].grid(alpha=0.3)
        plt.colorbar(sc, ax=axes[1], label='Пористость')
    
    plt.tight_layout()
    plt.show()

## 10. Корреляционная матрица

In [ ]:
corr_cols = [col for col in ['Porosity', 'Sw', 'Gamma', 'Rho'] if col in df.columns]

if len(corr_cols) >= 2:
    print("="*70)
    print("5️⃣  КОРРЕЛЯЦИИ МЕЖДУ ПАРАМЕТРАМИ")
    print("="*70)
    
    print("\nОбщая корреляционная матрица:")
    corr_matrix = df[corr_cols].corr()
    print(corr_matrix.round(3))
    
    # Тепловая карта
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm',
                square=True, linewidths=1, cbar_kws={"shrink": 0.8},
                vmin=-1, vmax=1)
    plt.title('Корреляционная матрица', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

## 11. Оценка шума

In [ ]:
print("="*70)
print("6️⃣  ШУМЫ И ЭФФЕКТЫ СГЛАЖИВАНИЯ")
print("="*70)

if 'Gamma' in df.columns:
    window = 5
    smooth = df['Gamma'].rolling(window=window, center=True).mean()
    noise = df['Gamma'] - smooth
    noise_std = noise.std()
    signal_std = df['Gamma'].std()
    snr = signal_std / noise_std if noise_std > 0 else float('inf')
    
    print(f"\nГамма-каротаж:")
    print(f"  Уровень шума (σ):  {noise_std:.2f} API")
    print(f"  SNR:               {snr:.1f}")
    
    config['gamma_noise_std'] = float(noise_std)

if 'Rho' in df.columns:
    window = 5
    smooth = df['Rho'].rolling(window=window, center=True).mean()
    noise = df['Rho'] - smooth
    noise_std = noise.std()
    
    print(f"\nПлотность:")
    print(f"  Уровень шума (σ):  {noise_std:.3f} г/см³")
    
    config['rho_noise_std'] = float(noise_std)

print(f"\nРекомендации для генератора:")
print(f"  Окно инверсии:     5-10 точек")
print(f"  Сила инверсии:     0.5-0.7")

## 12. Сохранение конфигурации

In [ ]:
print("="*70)
print("💾 СОХРАНЕНИЕ КОНФИГУРАЦИИ")
print("="*70)

output_file = 'calibrated_config.json'

# Добавление метаданных
config['metadata'] = {
    'source_file': input_file,
    'n_points': len(df),
    'n_lithologies': len(df['Lithology'].unique()),
    'depth_range': [float(df['Depth'].min()), float(df['Depth'].max())] if 'Depth' in df.columns else None
}

# Сохранение
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Конфигурация сохранена в: {output_file}")
print(f"📊 Размер конфига: {len(json.dumps(config))} байт")

print("\n📋 Содержимое конфига:")
for key in config.keys():
    print(f"  • {key}")

print("\n✅ Калибровка завершена!")
print("\n💡 Используйте 'calibrated_config.json' для генерации синтетических скважин")

## 13. Просмотр конфигурации

In [ ]:
print("📄 ФИНАЛЬНАЯ КОНФИГУРАЦИЯ:\n")
print(json.dumps(config, indent=2, ensure_ascii=False))

## 📝 Следующие шаги

1. **Используйте созданный конфиг** в генераторе синтетических скважин
2. **Загрузите параметры** в `app.py` или `well_generator.ipynb`
3. **Сравните** сгенерированные данные с реальными
4. **Скорректируйте** параметры при необходимости

### Как использовать конфиг:

```python
import json

# Загрузка конфига
with open('calibrated_config.json', 'r') as f:
    config = json.load(f)

# Использование параметров
lithology_freq_range = config['lithology_freq_range']
series_length_range = config['series_length_range']
lithology_properties = config['lithology_properties']
transition_matrix = np.array(config['transition_matrix'])
```

---
🔬 **Калибровка генератора синтетических скважин** | 2025